In [39]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow
import os
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img , img_to_array

In [28]:
import os

data_dir = r"C:\Users\mohit\Downloads\mixed\mixed"
image_files = []
labels = []
directory = {'none': 0, 'red': 1, 'yellow': 2, 'green': 3}

for class_folder in os.listdir(data_dir):
    class_folder_path = os.path.join(data_dir, class_folder)
    if os.path.isdir(class_folder_path) and class_folder in directory:
        class_label = directory[class_folder]

        class_image_files = [file for file in os.listdir(class_folder_path) if file.endswith('.jpg')]

        labels.extend([class_label] * len(class_image_files))

        image_files.extend([os.path.join(class_folder_path, file) for file in class_image_files])


Spliting the data

In [32]:
x_train, x_test, y_train , y_test = train_test_split(image_files,labels,test_size=0.2,random_state=42)

In [55]:
len(x_train)

1980

In [33]:
out=[i for i in labels if i==0]
out1=[i for i in labels if i==1]
out2=[i for i in labels if i==2]
out3=[i for i in labels if i==3]

In [34]:
print(len(out), len(out1) , len(out2) , len(out3))

1593 592 50 240


model will genrelize better without class imbalance. preprocessing the image , Applying RandomOverSampling

In [40]:
img_width, img_height = 224, 224

def preprocess_image(file_path, label):
    img = load_img(file_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = img_array / 255.0 
    return img_array, label

x_train_processed = []
for img_path, label in zip(x_train, y_train):
    img_array, label = preprocess_image(img_path, label)
    x_train_processed.append(img_array)

x_train_processed = np.array(x_train_processed)
y_train_categorical = to_categorical(y_train) 


x_test_processed = []
for img_path, label in zip(x_test, y_test):
    img_array, label = preprocess_image(img_path, label)
    x_test_processed.append(img_array)

x_test_processed = np.array(x_test_processed)
y_test_categorical = to_categorical(y_test)

In [53]:
x_train_flattened = x_train_processed.reshape(x_train_processed.shape[0], -1)
ros=RandomOverSampler(random_state=42)
x_resampled, y_resampled = ros.fit_resample(x_train_flattened, y_train_categorical)

In [56]:
original_shape = x_train_processed.shape[1:]
x_resampled_original_shape = x_resampled.reshape(-1, *original_shape)

In [57]:
x_resampled_original_shape.shape

(5072, 224, 224, 3)

In [64]:
len(y_resampled)

5072

In [63]:
x_train_processed.shape

(1980, 224, 224, 3)

final images for training  x_resampled_original_shape and  y_resampled

In [66]:
type(x_resampled_original_shape)

numpy.ndarray

In [67]:
type(y_resampled)

numpy.ndarray

In [69]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=None)

train_generator = datagen.flow(x_resampled_original_shape, y_resampled, batch_size=100, shuffle=True)
val_generator = datagen.flow(x_test_processed, y_test_categorical, batch_size=100, shuffle=False)

In [76]:
len(val_generator)

5

In [77]:
model=Sequential()
        
model.add(Conv2D(32,(3,3),padding='same',input_shape=(224,224,3),activation=tensorflow.nn.relu))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
        
model.add(Conv2D(64,(3,3),padding='same',activation=tensorflow.nn.relu))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
model.add(Flatten())

model.add(Dropout(rate=0.5))
        
model.add(Dense(528,activation=tensorflow.nn.relu))
model.add(Dense(4,activation=tensorflow.nn.softmax))
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

checkpoint_path = "saved_models/"
checkpoint_file = checkpoint_path + "model_epoch_{epoch:02d}.ckpt"
checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(filepath=checkpoint_file,
                                                                 save_weights_only=False,
                                                                 save_best_only=True,
                                                                 monitor='val_loss',
                                                                 verbose=1)
        
history= model.fit(train_generator,epochs=30,verbose=1,validation_data=val_generator , callbacks=[checkpoint_callback],steps_per_epoch= 51, validation_steps=5)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 200704)            0         
                                                                 
 dropout (Dropout)           (None, 200704)           

INFO:tensorflow:Assets written to: saved_models\model_epoch_01.ckpt\assets


51/51 [==============================] - 132s 3s/step - loss: 2.8114 - accuracy: 0.3829 - val_loss: 1.2742 - val_accuracy: 0.2909
Epoch 2/30
51/51 [==============================] - ETA: 0s - loss: 1.1652 - accuracy: 0.4852
Epoch 2: val_loss improved from 1.27416 to 1.05022, saving model to saved_models\model_epoch_02.ckpt
INFO:tensorflow:Assets written to: saved_models\model_epoch_02.ckpt\assets


INFO:tensorflow:Assets written to: saved_models\model_epoch_02.ckpt\assets


51/51 [==============================] - 129s 3s/step - loss: 1.1652 - accuracy: 0.4852 - val_loss: 1.0502 - val_accuracy: 0.5152
Epoch 3/30
51/51 [==============================] - ETA: 0s - loss: 1.0316 - accuracy: 0.5657
Epoch 3: val_loss improved from 1.05022 to 0.90116, saving model to saved_models\model_epoch_03.ckpt
INFO:tensorflow:Assets written to: saved_models\model_epoch_03.ckpt\assets


...

51/51 [==============================] - 287s 6s/step - loss: 0.2718 - accuracy: 0.8908 - val_loss: 0.2672 - val_accuracy: 0.8747
Epoch 25/30
51/51 [==============================] - ETA: 0s - loss: 0.2661 - accuracy: 0.8941
Epoch 25: val_loss improved from 0.26723 to 0.23224, saving model to saved_models\model_epoch_25.ckpt
INFO:tensorflow:Assets written to: saved_models\model_epoch_25.ckpt\assets


INFO:tensorflow:Assets written to: saved_models\model_epoch_25.ckpt\assets


51/51 [==============================] - 283s 6s/step - loss: 0.2661 - accuracy: 0.8941 - val_loss: 0.2322 - val_accuracy: 0.9071
Epoch 26/30
51/51 [==============================] - ETA: 0s - loss: 0.2659 - accuracy: 0.8943
Epoch 26: val_loss did not improve from 0.23224
51/51 [==============================] - 280s 5s/step - loss: 0.2659 - accuracy: 0.8943 - val_loss: 0.2769 - val_accuracy: 0.8707
Epoch 27/30
51/51 [==============================] - ETA: 0s - loss: 0.2558 - accuracy: 0.8933
Epoch 27: val_loss did not improve from 0.23224
51/51 [==============================] - 290s 6s/step - loss: 0.2558 - accuracy: 0.8933 - val_loss: 0.2723 - val_accuracy: 0.9010
Epoch 28/30
51/51 [==============================] - ETA: 0s - loss: 0.2442 - accuracy: 0.9032
Epoch 28: val_loss did not improve from 0.23224
51/51 [==============================] - 288s 6s/step - loss: 0.2442 - accuracy: 0.9032 - val_loss: 0.3354 - val_accuracy: 0.8586
Epoch 29/30
51/51 [==============================] 

INFO:tensorflow:Assets written to: saved_models\model_epoch_30.ckpt\assets


51/51 [==============================] - 331s 6s/step - loss: 0.2538 - accuracy: 0.9006 - val_loss: 0.2317 - val_accuracy: 0.8949
